In [1]:
import os
import time
import json
import pandas as pd
import torch
import loader
import utils
import preprocess_data
from config import config_base
from config import config_r_net
from config import config_match_lstm
from config import config_bi_daf
from config import config_ensemble
from modules import match_lstm
from modules import r_net
from modules import bi_daf

# config = config_match_lstm.config
# config = config_r_net.config
config = config_bi_daf.config
# config = config_ensemble.config

In [3]:
lang = loader.load_vocab(config.vocab_path)

In [4]:
embedding_np = loader.load_w2v(config.embedding_path)

In [9]:
train_data = loader.load_data(config.train_df, lang)

In [10]:
train_loader = loader.build_loader(
        dataset=train_data,
        batch_size=config.batch_size,
        shuffle=True,
        drop_last=True
    )

In [13]:
for batch in train_loader:
    s = batch[2]
    e = batch[3]
    for ss, ee in zip(s, e):
        if ss > ee:
            print(ss)
            print(ee)


tensor(5)
tensor(4)


In [14]:
import jieba

In [15]:
jieba.lcut('')

[]